# 默认代码

In [ ]:
from IPython.display import display
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd                 # pd._libs.tslibs.timestamps.Timestamp
import torch

In [ ]:
from datetime import datetime       # datetime.datetime
from datetime import timedelta
import time

# 路径拼接
>- 当前 .py 文件或 .ipynb 文件所在的文件夹目录
>- 两种方式见文件 D:\_Code\LJR_HMS\MyFirstPyCharmPath\TEST.py
>  - 使用按钮运行代码（同 jupyter notebook）
>  - 使用终端运行代码（工作路径为当前项目目录）
>- 三杆路径 `_ _ _DATA`

In [ ]:
import os

# 属性
>- DataFrame 的属性

In [ ]:
def shp(_):
    if type(_) == pd.core.frame.DataFrame:
        ss = f'({_.shape[0]:5d} , {_.shape[1]:5d})'
        print(f'{ss:<25s}   {type(_)}')
    elif type(_) == torch.Tensor or type(_) == np.ndarray:
        print(f'{str(_.shape):<25s}   {type(_)}')
    elif type(_) == list or type(_) == tuple:
        print(f'{len(_):<25d}   {type(_)}')
    else:
        print(f'type {type(_)} has no shape!')

def other_shp(_):
    if type(_) == pd.core.frame.DataFrame:
        print(f'line = {len(_):5d}' , end = '    ')       # 行
        print(f'size = {_.size:5d}')                      # 行 × 列


def col(data):
    cols = data.columns.values
    # print(f'{str(type(cols)):<25s}     {cols}')
    print('--------------------------------------------------------')
    for c in cols:
        print(f'{str(type(data.iloc[0][c])):<25s}     {c:^15s}')
    print('--------------------------------------------------------')
    return cols

# 搜索
>- 在 DataFrame 的指定列 col 中搜索指定列值的行

In [ ]:
# 1、ch == int
# 2、ch in String
# 3、ch in List
def exist(ch , src):
    if isinstance(ch , int) and isinstance(src , int):
        return ch == src
    else:
        return ch in src

In [ ]:
# cols 为 DataFrame 的一列或多列 DataFrame
# src 为 cols 中的列名，在该列中搜索 ch
def find_ch_in_cols(cols , src , ch):
    res = pd.DataFrame()
    res[cols.columns] = cols
    res['exist'] = res.apply(lambda row : exist(ch , row[src]) , axis = 1)
    return res[res['exist'] == True]

In [ ]:
# 按 chs 的顺序拼接行
# 注意，若 chs 里面有 ch1 和 ch2 都出现在同一行中，则该行会重复拼接到 res 中，可依据隐式行索引观察到
def find_chs_in_cols(cols , src , chs):
    res = pd.DataFrame()
    for ch in chs:
        res = pd.concat([res , find_ch_in_cols(cols , src , ch)])
        # reset_Idx(res)
    return res

# 重置隐式行索引
>- 拼接或排序后

In [ ]:
def reset_Idx(data):
    data.reset_index(drop = True , inplace = True)

# 比较
>- 比较 DataFrame 的两列
>- [px 的使用]https://www.cnblogs.com/Mangnolia/p/14308471.html
>- [px 的使用]https://blog.csdn.net/qq_25443541/article/details/115603480
>- [px 的使用]https://blog.csdn.net/lemonbit/article/details/119012201

In [ ]:
import plotly.express as px

In [ ]:
# int == int
# String == String
def isSame(a, b):
    if a == b:
        return True
    else:
        return False

In [ ]:
# col1 为 DataFrame 的一列 Series
# col2 为 DataFrame 的一列 Series
def diff_Pie(col1 , col2):
    tmp = pd.DataFrame()
    tmp['one'] = col1
    tmp['two'] = col2
    tmp['same'] = tmp.apply(lambda row : isSame(row['one'] , row['two']) , axis = 1)
    
    cnt = tmp['same'].value_counts().reset_index()
    cnt.columns = ['same' , 'cnt']
    display(cnt)
    
    fig = px.pie(
        cnt ,
        names = 'same',        # 划分块，即每个扇形的意义
        values = 'cnt',        # 算占比，即每个扇形的占比
        title = f'Are the [{col1.name}] as same as the [{col2.name}] ?' , 
    )
    
    # 文本信息水平显示
    fig.update_traces(textposition = 'inside', 
                      textinfo = 'percent+label' , 
                      insidetextorientation = 'horizontal')
    
    fig.show()
    
    return tmp , cnt

# 统计

## 列值个数
>- DataFrame 指定列的列值统计情况
>- 自动忽略 NaN 值，且有 NaN 的那一列若为数值，则会自动转换为 float 类型

In [ ]:
# col 为 DataFrame 的一列 Series
def val_cnt(col):
    cnt = col.value_counts().reset_index()
    cnt.columns = [col.name , 'cnt']
    cnt = cnt.sort_values(by = 'cnt' , ascending = False)    # 按 cnt 降序排列
    cnt['proportion'] = cnt.apply(lambda row : f"{(row['cnt'] / sum(cnt['cnt'])):6.2%}" , axis = 1)
    return cnt

## 长度分布
>- 对于 int 列来说的列值统计分布情况

In [ ]:
# col 为 DataFrame 的一列 Series 且为 int 数值类型
def val_cnt_int(col):
    cnt = val_cnt(col)
    fig = px.bar(cnt , 
                 x = f'{col.name}', 
                 y = 'cnt' , 
                 title = f'The distribution of {col.name}')
    fig.show()

## 元素组成
>- 适用于 String 、List 列
>   - String 中的每一个字符的统计
>   - List 中每一个元素的统计

In [ ]:
# col 为 DataFrame 的一列 Series
def get_all_chs(col):
    
    chs = set()
    dic = dict()
    
    # Series  ==>  ndarray
    val = col.values
    
    # 对每一个列值
    for e in val:
        # 对该列值的每一个组成元素
        for ch in e:
            chs.add(ch)
            if ch in dic.keys():
                dic[ch] += 1
            else:
                dic[ch] = 1
    
    
    # Set  ==>  List  ==>  排序
    chs = list(chs)
    chs.sort()
    
    
    # Dictionary  ==>  DataFrame  ==>  按 cnt 列降序排序
    dicf = pd.DataFrame.from_dict(dic , orient = 'index').reset_index()
    dicf.columns = ['ch' , 'cnt']
    dicf = dicf.sort_values(by = 'cnt' , ascending = False)
    
    
    # List , Dictionary , DataFrame
    return chs , dic , dicf

# 存储一
>- 将 List 存储为 Text 文件
>- 除了读取，其他任何操作都不要在打开文件的情况下进行

In [ ]:
pre_path_text = r''

## 读
>- 指定的 fileName.txt 文件必须预先存在
>- 注意
>   - 用 Enter 、Space 、Tab 按键按出来的 \n 、空白、\t 会消除掉
>   - 用键盘打印出来的 \n \r \t 不会消除掉
>- 在 String 中
>   - 每个中文字符算 1 个长度
>   - 每个转义字符算 1 个长度

In [ ]:
# lines  ==  List  ==  ['第一行' , '第二行' , '第三行' , ...]
def read_txt(fileName):
    
    path = pre_path_text + f'\\{fileName}' + '.txt'
    
    with open(path , 'r' , encoding = 'utf-8') as file:
        lines = file.readlines()
        lines = [line.strip() for line in lines]                       # 去除每一行首尾空白 + \n + \t + \r
        lines = [line for line in lines if (len(line) != 0)]           # 去除空字符串
        lines = [line for line in lines if (line.isspace() != True)]   # 去除空白行
        return lines

## 写
>- 自动创建 fileName.txt 文件（追加也如是）
>- 覆盖追加 fileName.txt 文件（文件已经存在的情况下）

In [ ]:
# 直接覆盖
def cover_txt(lines , fileName):
    
    path = pre_path_text + f'\\{fileName}' + '.txt'
    
    with open(path , 'w' , encoding = 'utf-8') as file:
        for line  in lines:
            file.write(line)
            file.write('\n')         # 自带文件关闭功能，无需要再写 file.close()
    
    print(f"Create or Cover file '{path}'")

In [ ]:
# 末尾追加
def append_txt(lines , fileName):
    
    path = pre_path_text + f'\\{fileName}' + '.txt'
    
    with open(path , 'a' , encoding = 'utf-8') as file:
        file.write('\n')
        for line  in lines:
            file.write(line)
            file.write('\n')         # 自带文件关闭功能，无需要再写 file.close()
    
    print(f"Append file '{path}'")

# 存储二
>- 将 DataFrame 存储为 Excel 文件
>- 除了读取，其他任何操作都不要在打开文件的情况下进行
>- 注意：
>  - List 类型  ==>  存储  ==>  Excel  ==>  读取  ==> String 类型

In [ ]:
import openpyxl

## 路径前缀
>- 可以是绝对路径（从 D: 盘开始）
>- 可以是相对路径（相对当前运行该方法的 xxx.ipynb 文件的路径）

In [ ]:
pre_path_excel = r''

## 单 sheet
>- 自动创建 fileName.xlsx 文件（sheetName 自动命名 Sheet1）
>- 或者覆盖 fileName.xlsx 文件（文件已经存在的情况下）

In [ ]:
def save_ex(data , fileName):
    
    path = pre_path_excel + f'\\{fileName}' + '.xlsx'
    
    data.to_excel(path , index = False)
    
    print(f"Create or Cover file '{path}'")

In [ ]:
# 也可用于读取多 sheet 的文件，默认始终读取第一个 sheet
def read_ex(fileName):
    
    path = pre_path_excel + f'\\{fileName}' + '.xlsx'
    
    res = pd.read_excel(path)
    
    return res

## 读取 & 删除 sheet
>- 指定的 fileName.xlsx 文件必须预先存在
>- 若把最后一个 sheet 删除了，则会导致文件损坏

In [ ]:
def read_sheet(fileName):
    
    path = pre_path_excel + f'\\{fileName}' + '.xlsx'
    
    work_book = openpyxl.load_workbook(path)        # 用于删除 sheet
    sheet_names = work_book.sheetnames              # List
    
    return work_book , sheet_names

In [ ]:
def remove_sheet(fileName , sheetName):
    
    # 读取全部 sheet_names
    work_book , sheet_names = read_sheet(fileName)
    
    
    # 删除指定 sheetName
    # 保存其他 sheetName
    if sheetName in sheet_names:
        print(f'befor remove : {sheet_names}')
        path = pre_path_excel + f'\\{fileName}' + '.xlsx'
        work_book.remove(work_book[sheetName])
        work_book.save(path)
        print(f'after remove : {work_book.sheetnames}')
    else:
        print(f"No such sheet '{sheetName}'!")
        print(f'All sheet are {sheet_names}')

## 多 sheet
>- 指定的 fileName.xlsx 文件必须预先存在

In [ ]:
def save_ex_(data , fileName , sheetName):
    
    # 读取全部 sheet_names
    work_book , sheet_names = read_sheet(fileName)
    
    
    # 如果指定的 sheetName 已经存在，则需要先删除之，即逻辑更新
    if sheetName in sheet_names:
        print(f"requested sheetName = '{sheetName}' already exists , should be removed firstly")
        remove_sheet(fileName , sheetName)
    
    
    # 追加，不影响其他 sheet
    path = pre_path_excel + f'\\{fileName}' + '.xlsx'
    with pd.ExcelWriter(path , mode = 'a' , engine = 'openpyxl') as writer:
        data.to_excel(writer , sheetName , index = False)
    
    
    print(f"Save or Update file '{path}' as sheetName = '{sheetName}'")
    print(f'after update : {read_sheet(fileName)[1]}')

In [ ]:
# 指定读取具体哪一个 sheetName ，单 sheetName 也适用
def read_ex_(fileName , sheetName):
    path = pre_path_excel + f'\\{fileName}' + '.xlsx'
    res = pd.read_excel(path , sheetName)
    return res

# 存储三
>- 将 DataFrame 存储为 JSON 文件
>- 除了读取，其他任何操作都不要在打开文件的情况下进行
>- JSON 文件格式详解：
>  - 最外面是 `[]`
>  - 里面一个一个是 `{}`
>  - 再里面一个一个是 `key : value`
>- `orient = 'records'`
>  - DataFrame 的一行  ==  一个 `{}`
>  - 不包括行索引 index
>  - List 类型  ==>  存储  ==>  JSON  ==>  读取  ==> List 类型
>- 注意，存储的字符串带有 `/` 时，会在 `.json` 文件中显示为 `\/`

In [ ]:
pre_path_json = r''

In [ ]:
def save_json(data , fileName):
    
    path = pre_path_json + f'\\{fileName}' + '.json'
    
    data.to_json(path , orient = 'records' , force_ascii = False)     # 中文存储设置
    
    print(f"Create or Cover file '{path}'")

In [ ]:
def read_json(fileName):
    
    path = pre_path_json + f'\\{fileName}' + '.json'
    
    res = pd.read_json(path , orient = 'records')
    
    return res

# 字符串处理
>- line

## 汉字
>- 仅保留字符串中的中文字符
>- 仅去除字符串中的中文字符

In [ ]:
def isChinese(ch):
    if u'\u4e00' <= ch and ch <= u'\u9fa5':
        return True
    else:
        return False

In [ ]:
def include_Chinese(line):
    res = ''
    for ch in line:
        if isChinese(ch):
            res += ch
    return res

In [ ]:
def exclude_Chinese(line):
    res = ''
    for ch in line:
        if not isChinese(ch):
            res += ch
    return res

## 提取
>- 返回 True 的 ch 块
>- [方法作为方法的参数]https://www.cnblogs.com/meitian/p/6756665.html

In [ ]:
# 字符串 line 中是否包含数字
def Contain_Num(line):
    for ch in line:
        if ch.isdigit():
            return True
    return False

In [ ]:
def Needed_Num(ch):
    return ch.isdigit() or ch == '.' or ch == '%' or ch == '/' or ch == '(' or ch == ')'

In [ ]:
# 传入 Needed 方法，无需带 ()
# 调用 Needed 方法，需要带 ()
# String  ==>  List
def ExtractNeeded(line , Needed):
    
    # res  ==  List  ==  ['Neededs' , 'Neededs' , ...]
    res = []
    
    flag = True
    for i in range(len(line)):
        if flag and Needed(line[i]):
            l = i
            r = i + 1
            while r != len(line) and Needed(line[r]):
                r += 1
            res.append(line[l:r])
            flag = False
        elif not Needed(line[i]):
            flag = True
    return res

In [ ]:
ExtractNeeded('0.5甲有1人，乙有4.2人，是8%分，成绩高0.77分，那么，乙绩=多分．0.999%' , Needed_Num)

## 计数
>- 理应为 len(ExtractNeeded)

In [ ]:
# String  ==>  int
def CountNeeded(line , Needed):
    
    cnt = 0
    flag = True
    
    for ch in line:
        if Needed(ch):
            if flag:
                cnt += 1
                flag = False
        else:
            flag = True
    
    return cnt

In [ ]:
CountNeeded('0.5甲有1人，乙有4.2人，是8%分，成绩高0.77分，那么，乙绩=多分．0.999%' , Needed_Num)

## 分词
>- jieba.cut('...') 结果为 generator （可用于 for 循环或 list() 函数）
>- jieba.lcut('...') 结果为 List

In [ ]:
import jieba
import jieba.posseg as pseg

In [ ]:
def Cut(line):
    return jieba.lcut(line)

In [ ]:
# List 、String 、Tuple ==>  len()  ==>  int
def Len(line):
    return len(line)

In [ ]:
# 抽取动词
# String  ==>  List verb.
def Cut_Verbs(line):
    words = pseg.cut(line)
    return [word for word , flag in words if flag == 'v']

In [ ]:
# 分词 + 词性标注
for word , flag in pseg.cut('我练'):
    print(f'{type(word)}   {word:3s}   {flag:3s}{type(flag)}')

## 停用词
>- 先分词后再去除停用词
>- List  ==>  过滤停用词  ==>  List

In [ ]:
def StopWords():
    s = set()
    with open(r'___DATA\Stop Word List\Common Stop Words.txt' , 'r' , encoding = 'utf-8') as f:
        for line in f:
            s.add(line.strip())
    return s

In [ ]:
def Filter(L):
    return [word for word in L if (word not in SW)]

In [ ]:
SW = StopWords()
print(f'{type(SW)}   {len(SW)}')
print('' in SW)

## 正则表达式

In [ ]:
import re

# List

## 两种差集
>- List  -  List  ==  List

In [ ]:
one = [1 , 2 ,     4 , 4 , 4 , 3 , 5]
two = [1 , 2 , 2 , 4 , 4]

In [ ]:
# one 中独有的
def diff_set(one , two):
    return [item for item in one if (item not in two)]

diff_set(one , two)

In [ ]:
# one 中独有的 + one 在二者交集上多出来的
def diff_list(one , two):
    t = two.copy()       # 避免修改原 two 的内容
    res = []
    for num in one:
        if num in t:
            t.remove(num)
        else:
            res.append(num)
    return res

diff_list(one , two)

In [ ]:
print(one)
print(two)

## 随机 & 打乱

In [1]:
from copy import deepcopy
import random

In [ ]:
print(random.random())             # [0 , 1)    浮点数
print(random.randint(9 , 10))      # [9 , 10]   整数
print(random.uniform(9 , 10))      # [9 , 10]   浮点数
print(random.choice([1 , 2 , 3]))  # 随机抽取一个元素，List 、Tuple
print(random.choice('我a爱b你'))   # 随机抽取一个字符，String


# 随机采样，不影响原 List
l = [1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10]  
s = random.sample(l , 5)
print(s) 
print(l)


# 随机打乱，会影响原 List
random.shuffle(l)
print(l)

In [ ]:
# 不改变原 List 的随机 & 完全 & 打乱
# 注意：此方法不适用于 Tuple 、String ，因为该二者是内容不可变对象
def shuffle(l):
    # 复制原 List
    temp_l = deepcopy(l)
    n = len(temp_l)
    
    # while 循环可以理解为还剩 n 个元素没有调换
    while(n):
        # 当前调换 temp_l[n - 1] 元素
        n -= 1
        if n == 1:
            # 调换 temp_l[0] 和 temp_l[1]
            temp_l[n] , temp_l[0] = temp_l[0] , temp_l[n]
            break
        else:
            # 从 temp_l[0 , n - 2] 中随机抽取元素调换
            i = random.randint(0 , n - 1)
            temp_l[n] , temp_l[i] = temp_l[i] , temp_l[n]
    return temp_l

# 时间条

In [ ]:
def data_bar():
    now = datetime.now().strftime('%Y-%m-%d')
    print(f"{'=============' * 6} {now} ")

def time_bar():
    now = datetime.now().strftime('  %H:%M:%S')
    print(f"{'=============' * 6} {now} ")
    
def data_time_bar():
    now = datetime.now().strftime('%Y-%m-%d  %H:%M:%S')
    print(f"{'=============' * 6} {now} ")

In [ ]:
#  datetime.datetime --> String
print(type(datetime.now()))
print(type(datetime.now().strftime('%Y-%m-%d %H:%M:%S')))


# 注意不需要前缀 datetime.
if type(datetime.now()) is datetime:
    print('1')
if type(datetime.now()) == datetime:
    print('1')
if type(datetime.now().strftime('%Y-%m-%d %H:%M:%S')) is str:
    print('2')
if type(datetime.now().strftime('%Y-%m-%d %H:%M:%S')) == str:
    print('2')

In [ ]:
#  datetime.datetime --> pd._libs.tslibs.timestamps.Timestamp --> String
print(type(datetime.now()))
print(type(pd.Timestamp(datetime.now())))
print(type(pd.Timestamp(datetime.now()).strftime('%Y-%m-%d %H:%M:%S')))


# 注意 pandas 缩写 pd
if type(pd.Timestamp(datetime.now())) is pd._libs.tslibs.timestamps.Timestamp:
    print('1')
if type(pd.Timestamp(datetime.now())) == pd._libs.tslibs.timestamps.Timestamp:
    print('1')

# 时间秒
>- time.time() 为 float 类型

In [ ]:
# start 也应该为 time.time()
def get_time_dif(start):
    end = time.time()
    dif = end - start
    return timedelta(seconds = int(round(dif)))

# StanfordCoreNLP
>- [使用]https://zhuanlan.zhihu.com/p/137226095
>- [下载]https://stanfordnlp.github.io/CoreNLP/
>- pip show stanfordcorenlp 中版本为 3.9.1.1 所以对应的解压包.zip和中文包.jar需要下载旧版本 3.9.2

In [ ]:
# from stanfordcorenlp import StanfordCoreNLP

# 英文处理
# nlp = StanfordCoreNLP(r'___DATA\stanford-corenlp-4.5.1')

# 中文处理
# nlp_ch = StanfordCoreNLP(r'___DATA\stanford-corenlp-full-2018-10-05', lang = 'zh')